# Opening a Coffee Shop in Seattle

## Introduction

   Seattle is a Mecca for coffee. There are many options for customers and fierce competition among providers.

   This notebook will be looking to determine which neighborhood would be the optimal place to open a new cafe. It will leverage the data provided by the Foursquare API among other data sources to compare various possible neighborhoods. 

## Data Required

We will start by developing a dataset to represent the various neighborhoods of Seattle, potentially divided by zip code.

The most important data will be the locations of coffee shops alrady in operation. We want to find an area that is not overly saturated. This will help limit the areas that are considered. Starbucks locations will be of particular interest as Seattle is where this beheamoth was founded. 

Additionally, we will want to analyze other locations that are nearby. Foot traffic is incredibly important to the viability of a coffee shop. What areas have other venues that will attract people to the area?

If possible, we will want to find another data source that will give us insight into the residential versus commercial nature of various neighborhoods. Other valueable data could be statistics covering income. If we cannot find income data that is adequate, housing prices like those found in an earlier module could serve as an effective approximation. 